In [1]:
user_path='../../user_data/'
import pandas as pd
df=pd.read_pickle(user_path+'r_list_binn_0527.pkl')
df=df[0]

pred_df=df

pred_df=pred_df.loc[pred_df['rank'] <=50 ]
preds={}
for user in pred_df.user_id.unique():
    last = len(pred_df.loc[pred_df.user_id == user ])
    if last == 50:
        index_list = pred_df.loc[pred_df.user_id == user].index[:50] # it returns row labels
        preds[int(user)] = pred_df.loc[index_list].item_id_pred.astype(int).to_list()
    else:
        index_list = pred_df.loc[pred_df.user_id == user].index[:last]
        preds[int(user)] = pred_df.loc[index_list].item_id_pred.astype(int).to_list()+top50_items[last:]

user_to_be_filled= answers[p].keys() - preds.keys()
for user in user_to_be_filled:
    preds[user] = top50_items